# MintPy for signal maps

In [15]:
# Import required packages
import os
import mintpy
from pathlib import Path
from typing import List, Union
from osgeo import gdal

In [16]:
orbit_list = ['AT137']
year_list = ['2017', '2018', '2019', '2020', '2021']

## clip files to common extent

In [17]:
def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

In [18]:
def clip_hyp3_products_to_common_overlap(data_path: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """
    
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase_ERA5.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:
        print(f'working on {extension}') 
        for file in data_path.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

## Mintpy

In [19]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [20]:
# function to clip files and run mintpy for multiple years 
def mintpy_multiyear(orbit_list, year_list, clip=True, mintpy=True, clean_clip=True):
    # hardcoded paths for now 
    home_path_d = '/mnt/d/indennt'
    home_path = '/mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data'
    for orbit in orbit_list:
        for year in year_list:
            print(f'working on {orbit}, {year}')
            data_path = f'{home_path_d}/hyp3_app/{orbit}/{year}'
            mintpy_path = f'{home_path}/signal_mintpy/{orbit}/mintpy_{year}_ERA5'
            mintpy_path_d = f'{home_path_d}/mintpy_app/{orbit}/'

            if clip==True:
                # identify and crop to common overlap
                print('identifying common overlap')
                dem_files = Path(data_path).glob('*/*_dem.tif')
                overlap = get_common_overlap(dem_files)
                print('clipping to common overlap')
                clip_hyp3_products_to_common_overlap(Path(data_path), overlap)

            # make output dir for mintpy
            if not os.path.exists(mintpy_path):
                os.mkdir(mintpy_path)

            # write config file for mintpy
            CONFIG_TXT = f'''# vim: set filetype=cfg:
            ##----------------------------- hyp3 ---------------------##
            mintpy.load.processor        = hyp3
            ##---------interferogram datasets:
            mintpy.load.unwFile          = {data_path}/*/*{year}*unw_phase_ERA5_clipped.tif
            mintpy.load.corFile          = {data_path}/*/*{year}*corr_clipped.tif
            ##---------geometry datasets:
            mintpy.load.demFile          = {data_path}/*/*{year}*dem_clipped.tif
            mintpy.load.incAngleFile     = {data_path}/*/*{year}*lv_theta_clipped.tif
            mintpy.load.waterMaskFile    = {data_path}/*/*{year}*water_mask_clipped.tif

            mintpy.deramp                = linear
            mintpy.reference.lalo        = auto
            mintpy.troposphericDelay.method  = no

            mintpy.compute.cluster    = local
            mintpy.compute.numWorker  = 6
            '''

            os.chdir(mintpy_path)
            config_file = f'{mintpy_path}/{year}_Sen{orbit}.txt'
            write_config_file(config_file, CONFIG_TXT, mode='w')

            if mintpy==True:
                # run mintpy
                print('starting mintpy')
                !smallbaselineApp.py --dir {mintpy_path} {config_file}
                print('moving outputs to drive')
                !cp -r $mintpy_path $mintpy_path_d && rm -R $mintpy_path

            if clean_clip==True:
                # remove clipped files
                print('removing clipped files')
                clipped_files = f'{data_path}/*/*_clipped.tif'
                !rm {clipped_files}

In [21]:
mintpy_multiyear(orbit_list, year_list, clip=True, mintpy=True, clean_clip=True)

working on AT137, 2017
identifying common overlap
clipping to common overlap
working on _water_mask.tif
working on _corr.tif
working on _unw_phase_ERA5.tif
working on _dem.tif
working on _lv_theta.tif
working on _lv_phi.tif
write configuration to file: /mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data/signal_mintpy/AT137/mintpy_2017_ERA5/2017_SenAT137.txt
starting mintpy

___________________________________________________________

  /##      /## /##             /##     /#######
 | ###    /###|__/            | ##    | ##__  ##
 | ####  /#### /## /#######  /######  | ##  \ ## /##   /##
 | ## ##/## ##| ##| ##__  ##|_  ##_/  | #######/| ##  | ##
 | ##  ###| ##| ##| ##  \ ##  | ##    | ##____/ | ##  | ##
 | ##\  # | ##| ##| ##  | ##  | ## /##| ##      | ##  | ##
 | ## \/  | ##| ##| ##  | ##  |  ####/| ##      |  #######
 |__/     |__/|__/|__/  |__/   \___/  |__/       \____  ##
                                                 /##  | ##
                                             

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 157432/157432 pixels   53s /    47s
[======================= 53%                       ] 161000/301435 pixels   53s /    47sconverting LOS phase unit from radian to meter
[======================= 53%                       ] 161200/301435 pixels   53s /    47s
FUTURE #4 complete. Time used: 68 seconds
[==================================================] 301435/301435 pixels   91s /   121s
converting LOS phase unit from radian to meter
[=====================>  43%                       ] 297600/689159 pixels   91s /   121s
FUTURE #5 complete. Time used: 106 seconds
[==================================================] 689159/689159 pixels  179s /     3s
converting LOS phase unit from radian to meter

FUTURE #6 complete. Time used: 196 seconds
close dask client
close dask cluster
------- finished parallel processing -------


--------------------------------------------------
open  HDF5 file timeseries.h5 in a mode
writing dataset /times

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 767020/767020 pixels   369s /   182s
converting LOS phase unit from radian to meter
[======================= 67% =====>                ] 792600/1185340 pixels  369s /   182s
FUTURE #2 complete. Time used: 386 seconds
[==================================================] 1126954/1126954 pixels  485s /    30s
[======================= 93% =================>    ] 1103000/1185340 pixels  485s /    36sconverting LOS phase unit from radian to meter
[======================= 93% =================>    ] 1103200/1185340 pixels  485s /    36s
FUTURE #3 complete. Time used: 501 seconds
[==================================================] 1185340/1185340 pixels  506s /    10s
[==================================================] 1173600/1185340 pixelsconverting LOS phase unit from radian to meter
[==================================================] 1173800/1185340 pixels
FUTURE #4 complete. Time used: 523 seconds
[===================================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 862246/862246 pixels   397s /   146s
[======================= 73% =======>              ] 862000/1185340 pixels  397s /   146sconverting LOS phase unit from radian to meter
[======================= 73% =======>              ] 867400/1185340 pixels  397s /   147s
FUTURE #2 complete. Time used: 413 seconds
[==================================================] 1185340/1185340 pixels  502s /    10s
converting LOS phase unit from radian to meter
[==================================================] 1172400/1185340 pixels  502s /    10s
FUTURE #3 complete. Time used: 518 seconds
[==================================================] 1185340/1185340 pixels  504s /    10s
converting LOS phase unit from radian to meter
[==================================================] 1171800/1185340 pixels
FUTURE #4 complete. Time used: 521 seconds
[==================================================] 1185340/1185340 pixels 
converting LOS phase unit from radi